In [3]:
# Install minimal dependencies (`torch`, `transformers`, `timm`, `tokenizers`, ...)
# > pip install -r https://raw.githubusercontent.com/openvla/openvla/main/requirements-min.txt
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

import torch

# Load Processor & VLA
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b", 
    attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
    torch_dtype=torch.bfloat16, 
    low_cpu_mem_usage=True, 
    trust_remote_code=True
).to("cuda:0")

image_path = '/home/xilun/openvla/test_image.png'
# load_image 
def get_from_camera(image_path: str) -> Image.Image:
    return Image.open(image_path)
# Grab image input & format prompt
image: Image.Image = get_from_camera(image_path)
prompt = "In: What action should the robot take to {Lift the blue cube}?\nOut:"

# Predict Action (7-DoF; un-normalize for BridgeData V2)
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

print (action) 

# # Execute...
# robot.act(action, ...)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.86it/s]


[ 0.00471205 -0.00171916 -0.00569757  0.00305409 -0.01819708  0.00799228
  0.99607843]


In [1]:
# import openai
from openai import OpenAI
import os
import time

def open_ai_call_with_retry(model_name, messages, temperature=0.0, max_tokens=300):
    reset_trigger_phrases = ["CHOICE", "NUM"]
    success = False
    while not success:
        try:
            completion = client.chat.completions.create(
                model=model_name,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            success = True
            for reset_trigger_phrase in reset_trigger_phrases:
                if reset_trigger_phrase in completion.choices[0].message.content:
                    success = False
        except Exception as e:
            print("OpenAI API call issue, re-trying..." + str(e) + "\n")
            time.sleep(5)
    return completion

model = "gpt-4"
client = OpenAI(api_key = 'sk-proj-auJnJ5mPUzDvz39YhZhIT3BlbkFJMOvvbkfTcvPzIaGhxCwP')


messages = [
    {
        "role": "system",
        "content": "You are an intelligent robot planner that can understand human needs based on their reactions\
            and language to your behavior. You also pay attention to the user command history and the user's reaction to \
                your behavior, so that you know the user's preference when you generate new plans.",
    },
    {
        "role": "user",
        "content": "I have three drinks in front of me. One coke, one water, and one coffee.\
            Now I am thristy, what should i do?",
    },
]

temperature = 0.05

# Initial completion
completion = open_ai_call_with_retry(model, messages, temperature, max_tokens=2000)
if completion:
    response = completion.choices[0].message.content
    print(response)
    messages.append({"role": "assistant", "content": response})

# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    messages.append({"role": "user", "content": user_input})
    
    completion = open_ai_call_with_retry(model, messages, temperature, max_tokens=2000)
    if completion:
        response = completion.choices[0].message.content
        print ("User: " + user_input)
        print("ChatGPT: " + response)
        messages.append({"role": "assistant", "content": response})


Based on your statement, you should drink one of the beverages in front of you. Since you're thirsty, water would be the most hydrating choice. However, if you need a caffeine boost or prefer the taste, you might choose the coffee or coke.
User: Hmm, I kind of prefer sweet stuff
ChatGPT: In that case, you might enjoy the coke more as it is sweet. Please go ahead and drink the coke to quench your thirst.
User: OK sounds good, Also can you suggest me a snake? We have ginger candy, oreo, and a coffee candy
ChatGPT: Since you mentioned that you prefer sweet things, you might enjoy the Oreo cookies. They are sweet and can be a good snack to have along with your drink.
User: Can you generate detailed plan on how can I grab that? 
ChatGPT: Sure, here is a detailed plan for you:

1. Stand up from your current position.
2. Walk towards the location where the Oreo cookies are kept.
3. Extend your arm towards the Oreo packet.
4. Use your hand to grasp the packet of Oreos. Make sure your grip is f